<a href="https://colab.research.google.com/github/Yankee1231/Microprocessor_practice/blob/main/Microprocessor_Week4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Microprocessor 6주차
-----
### assebly C++ 동치 코드 예
어셈블리
```
;Define pull-ups and set outputs high
;Define directions for port pins
Idi r16,(1<<PB7)|(1<<PB6)|(1<<PB1)|(1<<PB0)
Idi r17,(1<<DDB3)|(1<<DDB2)|(1<<DDB1)|(1<<DDB0)
out PORTB,r16
out DDRB,r17
;Insert nop for synchronization
nop
;Read port pins
in r16, PINB
```
C++  
```cpp
/* Define pull-ups and set outs high */
/* Define direction for port pins */
PORTB = (1<<PB7)|(1<<PB6)|(1<<PB1)|(1<<PB0);
DDRB = (1<<DDB3)|(1<<DDB2)|(1<<DDB1)|(1<<DDB0);
/* Inser nop for synchronization*/
__no_operation();
/* Read port pins*/
i = PINB;
```


### E.g. 1: LED On
```cpp
#include<avr/io.h>

int main(void)
{
  MCUCR = 0x00; //MCU Control Register

  DDRD = 0xff; //Set the D port output

  while(1)
  {
    //Main Loop
    DDRD = 0x00; //LED On
  }
}
```

### E.g.2: LED Blink
```cpp
#include <avr/io.h>

void delay_us(unsigned char time_us)
void delay_ms(unsigned int time_ms)

int main(void)
{
  MCUCR = 0x00;
  DDRD = 0xff;

  while(1)
  {
    DDRD = 0x00;    // All LEDs are On
    delay_ms(500);
    DDRD = 0xff;    // All LEDs are Off
    delay_ms(500);
  }
}

void delay_us(unsigned char time_us)
{
  unsigned char i;
  for(i=0;i<time_us;i++)
  {
    asm volatile("PUSH R0"); //2clock
    asm volatile("POP R0");  //2clock
    asm volatile("PUSH R0"); //2clcok
    ams volatile("POP R0");  //2clock
    asm volatile("PUSH R0"); //2clock
    asm volatile("POP R0");  //2clock
  }
}

void delay_ms(unsigned int time_ms)
{
  unsigned int i;
  for(i=0;i<time_ms;i++)
  {
    delay_us(250);
    delay_us(250);
    delay_us(250);
    delay_us(250);
  }
}
```

### E.g. 3: LED moving On/Off
```cpp
#include <avr/io.h>
#include <avr/delay.h>

int main(void)
{
  char i, j, led;     
  DDRD = 0xf0;            //포트 D의 상위 4비트를 출력으로 설정
  while(1)
  {
    led = 0xef;           //LED 하위 1개를 ON 시키는 값
    for(i=0;i<3;i++)      //3번 반복
    {
      PORTD = led;        //LED 하위 1개 점등
      led = (led << 1);   //한 비트 씩 왼쪽으로 이동
      for(j=0;j<20;j++) delay_ms(10);
    }
    led = 0x7f;
    for(i=0;i<3;i++)
    {
      PORTD = led;        //LED 상위 1개 점등
      led = (led>>1) | 0x80; // 오른쪽으로 한 비트씩 이동
      for(j=0;j<20;j++) delay_ms(10);
    }
  }
}
```

### E.g 4: LED On/Off with PB1
```cpp
#include<avr/io.h>

unsigned char In;

int main(void)
{
  MCUCR = 0x00;
  DDRD = 0xff;
  DDRB = 0x00;

  while(1)
  {
    //Main Loop
    In = PibB & 0x02; // Port B-bit 1, Masking Method
    if(PinB == 0x02) PORTD = 0xff; // LED Off
    else             PORTD = 0x00; // LED On
  }
}
```

### E.g. 5: LED On/Off with PE
```cpp
#include<avr/io.h>
#include<avr/delay.h>
typedef unsigned char byte;
byte i, led;

void delay_200m()         // 0.2s delay
{
  byte k;
  for(k=0;k<20;k++)
    delay_ms(10);
}

void right()
{
  led = 0xef;             // LED
  for(i=0;i<3;i++)
  {
    PORTD = led;
    led = led << 1;       // Shift right
    delay_200m();
  }
}

void left()
{
  led = 0x7f;           
  for(i=0;i<3;i++)
  {
    PORTD = led;
    led = (led >> 1) | 0x80;
    delay_200m();
  }
}

void up()
{
  led = 0xe0;
  for(i=0;i<3;i++)
  {
    PORTD = led;
    led = led << 1;
    delay_200ms();
  }
}

void down()
{
  led = 0x7f;
  for(i=0;i<3;i++)
  {
    PORTD = led;
    led = led >> 1;
    delay_200m();
  }
}

void blink()
{
  PORTD = 0x00; // LED On
  delay_200m();
  PORTD = 0xff; // LED Off
  delay_200m();
}

int main(void)
{
  byte sw;
  DDRD = 0xff; //Port D high 4bit-output
  DDRE = 0x00; //Port E -input
  PORTE =0xf0; //pull UP- portE
  do{
    sw = PINE & 0xf0;
    switch(sw)
    {
      case 0xe0:
        right();
        break;
      case 0xd0;
        right();
        break;
      case 0xb0;
        right();
        break;
      case 0x70;
        right();
        break;
      default:
        blink():
        break;
    }
  }while(1);
}

```

### E.g. 6: FND Display1
```cpp
#include<avr/io.h>
#include<avr/delay.h>
typedef unsigned char byte;
byte and_num[10] = {0x3f,0x06,0x5b, 0x4f, 0x66, 0x6d, 0x7d, 0x27, 0x7f, 0x6f};
byte and_sum[6] = {0x04, 0x08, 0x10, 0x20, 0x40, 0x80};
int main(void)
{
  byte i;
  DDRA = 0xff;
  DDRF = 0xfc;
  while(1)
  {
    for(i=0;i<6;i++)
    {
      PORTA = and_num[i];
      PORTF = and_loc[i];
      _delay_ms(2);
    }
  }
}

```

### E.g. 7:

```cpp
#include<avr/io.h>
#include<avr/delay.h>
typedef unsigned char byte;
byte snd_num[10] = {~};
byte snd_loc[6] = {~};
void int_part(void)
{
  DDRA = 0xff;
  DDRF = 0xfc;
}

void snd_set(byte num, byte loc)
{
  PORTA = snd_num[num];
  PORTF = ~snd_loc[loc];
  _delay_ms(2);
  PORTF = 0xff;
}

int main(void)
{
  byte i;
  int_part();
  while(1)
  {
    for(i=0;i<6;i++)
      snd_set(i,i);
  }
}
